In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import numpy as np
import time
from numba import vectorize, guvectorize

# Types

## Data type limits

int:  −(2n−1) to (2n−1 − 1)
uint: 0 to (2n − 1) 
float:

## Types in NumPy and NumBa
### NumPy:
https://numpy.org/doc/stable/user/basics.types.html
https://numpy.org/doc/stable/reference/arrays.dtypes.html

### NumBa:
https://numba.pydata.org/numba-doc/dev/reference/types.html#basic-types

In [46]:
npArr = np.random.randint(0, 3000, (10, 10))

convtype = np.uint8
convtype_info = np.iinfo(convtype)
np.clip(npArr, convtype_info.min, convtype_info.max, npArr)
npArr = npArr.astype(convtype)

# print(npArr) 
# print(npArr)


# NumBa stuff

## Vectorize

In [47]:
@guvectorize("(u4[:], u4, u4[:])", '(n),()->(n)')
def g(x, y, res):
    for i in range(x.shape[0]):
        res[i] = x[i] + y

a = np.arange(5, dtype='u4')
g(a, 2)

array([2, 3, 4, 5, 6], dtype=uint32)

# Game

## Imports

In [48]:
LAYERS = np.load("grid_example.npy")
rgbo_ref = grid_rgbo.mk_rgbo_ref()
print(rgbo_ref.shape)

(3, 5, 255, 4)


## Testing speeds

In [49]:
def mk_mix_rgbo(type, targ_str):
    type_str = f"({type}[:,:,:], {type}[:,:,:], {type}[:,:,:])"

    @guvectorize("(f8[:,:,:], f8[:,:,:], f8[:,:,:])",
                 '(i,j,x),(i,j,x)->(i,j,x)',
                 target=targ_str, nopython=True, cache=True)
    def mix_rgbo(rgbo1, rgbo2, rgbo_grid):
        r1 = rgbo1.shape[0]
        r2 = rgbo1.shape[1]
        for i in range(r1):
            for j in range(r2):
                rgbo_grid[i, j, 3] = 255
                rgbo_grid[i, j, :3] = rgbo1[i, j, :3] * \
                    (1-rgbo2[i, j, -1]/255) + \
                    rgbo2[i, j, :3]*(rgbo2[i, j, -1]/255)
    return mix_rgbo


def measure(rgbos1, rgbos2, iterations):
    times = []
    for _ in range(iterations):
        t0 = time.time()
        for _ in range(5):
            rgbo_map = mix_func(rgbos1, rgbos2)
        t1 = time.time()
        t = (t1-t0)*60
        times.append(t)
    avg = np.mean(times)
    print(avg)


types = ['i4', 'f8']
for type in types:
    rgbos1 = np.random.randint(0, 255, (80, 60, 4)).astype(type)
    rgbos2 = np.random.randint(0, 255, (80, 60, 4)).astype(type)
    mix_func = mk_mix_rgbo(type, 'cpu')
    measure(rgbos1, rgbos2, 10)


0.14838838577270508
0.1374063491821289


## Generalizing rgbo-makers

In [50]:
from src.game import grid_rgbo
from src.game import open_gl_tools


In [51]:
x = np.arange(LAYERS[0,0].shape[0], dtype='i8')
y = np.arange(LAYERS[0,0].shape[1], dtype='i8')
all_xy = np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])

In [52]:

rgbo_l01 = grid_rgbo.layer_to_rgbo(LAYERS, 0, 1, rgbo_ref)
rgbo_l02 = grid_rgbo.layer_to_rgbo(LAYERS, 0, 2, rgbo_ref)
rgbo_l11 = grid_rgbo.layer_to_rgbo(LAYERS, 1, 1, rgbo_ref)
rgbo_l20 = grid_rgbo.layer_to_rgbo(LAYERS, 2, 0, rgbo_ref)


l01_l02 = grid_rgbo.mix_2_rgbo_grids(rgbo_l01, rgbo_l02)
l01_l02_l11 = grid_rgbo.mix_2_rgbo_grids(l01_l02, rgbo_l11)
l01_l02_l11_l20 = grid_rgbo.mix_2_rgbo_grids(l01_l02_l11, rgbo_l20)

c_list = open_gl_tools.grid_to_clist(all_xy, l01_l02_l11_l20)

print(c_list.shape)

(115200,)


## More stuff?